In [1]:
import os
import tensorflow.keras as keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)

Notebook run using keras: 2.4.0


In [2]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input
import keras.models

# Change some parameters or model itself?

The ResNet50 Model from the previous file had:

- batch size of 32
- learning rate of 0.001
- no rescaling of the pixel components

In [20]:
# load dataset
IMAGE_SIZE = (160, 160)
BATCH_SIZE = 32
NUM_CLASSES = 4

data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split = 0.2,
    rescale=1./255.
)

train_ds = data_gen.flow_from_directory(
    'dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='training', 
)

val_ds = data_gen.flow_from_directory(
    'dataset',
    target_size=IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset='validation'
)

Found 3200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [3]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3)
]

In [5]:
# Make similar model with slight modifications

# make base model; it will be frozen
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
for layer in base_model.layers:
    layer.trainable = False
    
# put the trainable layers of the model
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(16, activation='relu')(x)
prediction_layer = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [7]:
model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.001),
                  metrics=['accuracy'])

In [9]:
history = model.fit(
    train_ds,
    steps_per_epoch=50,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
50/50 [==============================] - 130s 3s/step - loss: 1.4413 - accuracy: 0.2676 - val_loss: 1.3722 - val_accuracy: 0.3013
Epoch 2/5
50/50 [==============================] - 122s 2s/step - loss: 1.3694 - accuracy: 0.3254 - val_loss: 1.3604 - val_accuracy: 0.3313
Epoch 3/5
50/50 [==============================] - 120s 2s/step - loss: 1.3426 - accuracy: 0.3275 - val_loss: 1.3174 - val_accuracy: 0.4125
Epoch 4/5
50/50 [==============================] - 123s 2s/step - loss: 1.3211 - accuracy: 0.3767 - val_loss: 1.2818 - val_accuracy: 0.4162
Epoch 5/5
50/50 [==============================] - 121s 2s/step - loss: 1.2790 - accuracy: 0.4312 - val_loss: 1.2671 - val_accuracy: 0.4288


Looks like using the rescaling component in the preprocessor does not improve the model's accuracy.

The batch size will remain unchanged for simplicity. However, I will try different learning rate values and a bigger target size.

In [4]:
# load dataset
IMAGE_SIZE = (200, 200)
BATCH_SIZE = 32
NUM_CLASSES = 4

data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split = 0.2,
    #rescale=1./255.
)

train_ds = data_gen.flow_from_directory(
    'dataset',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    subset='training', 
)

val_ds = data_gen.flow_from_directory(
    'dataset',
    target_size=IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset='validation'
)

Found 3200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [5]:
# make base model; it will be frozen
base_model_2 = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
for layer in base_model_2.layers:
    layer.trainable = False
    
# put the trainable layers of the model
x = keras.layers.Flatten()(base_model_2.output)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(16, activation='relu')(x)
prediction_layer = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

Try with `lr = 0.01`.

In [ ]:
model = keras.models.Model(inputs=base_model_2.input, outputs=prediction_layer)

In [12]:
model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.01),
                  metrics=['accuracy'])

In [13]:
history = model.fit(
    train_ds,
    steps_per_epoch=50,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
50/50 [==============================] - 206s 4s/step - loss: 22.4180 - accuracy: 0.4271 - val_loss: 1.4538 - val_accuracy: 0.7638
Epoch 2/5
50/50 [==============================] - 200s 4s/step - loss: 0.9983 - accuracy: 0.8367 - val_loss: 1.0329 - val_accuracy: 0.8275
Epoch 3/5
50/50 [==============================] - 199s 4s/step - loss: 0.3867 - accuracy: 0.9150 - val_loss: 1.0858 - val_accuracy: 0.8450
Epoch 4/5
50/50 [==============================] - 199s 4s/step - loss: 0.2939 - accuracy: 0.9232 - val_loss: 0.8943 - val_accuracy: 0.8700
Epoch 5/5
50/50 [==============================] - 200s 4s/step - loss: 0.1311 - accuracy: 0.9647 - val_loss: 0.8275 - val_accuracy: 0.8537


Now try again with `lr = 0.001`

In [6]:
model2 = keras.models.Model(inputs=base_model_2.input, outputs=prediction_layer)

model2.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.001),
                  metrics=['accuracy'])

In [16]:
# this cell was run after training the model with lr = 0.01
history2 = model2.fit(
    train_ds,
    steps_per_epoch=50,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
50/50 [==============================] - 198s 4s/step - loss: 0.2287 - accuracy: 0.9502 - val_loss: 0.8247 - val_accuracy: 0.8750
Epoch 2/5
50/50 [==============================] - 192s 4s/step - loss: 0.0792 - accuracy: 0.9814 - val_loss: 0.8522 - val_accuracy: 0.8763
Epoch 3/5
50/50 [==============================] - 191s 4s/step - loss: 0.0882 - accuracy: 0.9857 - val_loss: 0.8383 - val_accuracy: 0.8712
Epoch 4/5
50/50 [==============================] - 190s 4s/step - loss: 0.0296 - accuracy: 0.9915 - val_loss: 0.9014 - val_accuracy: 0.8687


Accuracy is weirdly very high. Perhaps if we trying fitting this first...

In [25]:
# this cell was run without training the model with lr = 0.01 first
history2 = model2.fit(
    train_ds,
    steps_per_epoch=50,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
50/50 [==============================] - 202s 4s/step - loss: 2.5294 - accuracy: 0.5607 - val_loss: 0.5665 - val_accuracy: 0.8200
Epoch 2/5
50/50 [==============================] - 194s 4s/step - loss: 0.3724 - accuracy: 0.8955 - val_loss: 0.4591 - val_accuracy: 0.8462
Epoch 3/5
50/50 [==============================] - 194s 4s/step - loss: 0.2076 - accuracy: 0.9361 - val_loss: 0.4497 - val_accuracy: 0.8625
Epoch 4/5
50/50 [==============================] - 195s 4s/step - loss: 0.1087 - accuracy: 0.9663 - val_loss: 0.4129 - val_accuracy: 0.8662
Epoch 5/5
50/50 [==============================] - 198s 4s/step - loss: 0.0915 - accuracy: 0.9772 - val_loss: 0.4196 - val_accuracy: 0.8700


In [7]:
history2 = model2.fit(
    train_ds,
    steps_per_epoch=50,
    validation_data=val_ds,
    epochs=8,
    callbacks=callbacks
)

Epoch 1/8
50/50 [==============================] - 198s 4s/step - loss: 2.2601 - accuracy: 0.4570 - val_loss: 0.7229 - val_accuracy: 0.7675
Epoch 2/8
50/50 [==============================] - 192s 4s/step - loss: 0.5630 - accuracy: 0.8278 - val_loss: 0.5907 - val_accuracy: 0.8125
Epoch 3/8
50/50 [==============================] - 192s 4s/step - loss: 0.4107 - accuracy: 0.8799 - val_loss: 0.5677 - val_accuracy: 0.8350
Epoch 4/8
50/50 [==============================] - 193s 4s/step - loss: 0.2489 - accuracy: 0.9363 - val_loss: 0.5734 - val_accuracy: 0.8512
Epoch 5/8
50/50 [==============================] - 193s 4s/step - loss: 0.1946 - accuracy: 0.9427 - val_loss: 0.5256 - val_accuracy: 0.8612
Epoch 6/8
50/50 [==============================] - 194s 4s/step - loss: 0.1513 - accuracy: 0.9655 - val_loss: 0.5470 - val_accuracy: 0.8775
Epoch 7/8
50/50 [==============================] - 195s 4s/step - loss: 0.1310 - accuracy: 0.9688 - val_loss: 0.5532 - val_accuracy: 0.8662
Epoch 8/8
50/50 [===

In [8]:
model2.save('my_model_2')

INFO:tensorflow:Assets written to: my_model_2/assets
